- update common class again
- update param tuning method *

# common class

In [1]:
import sys
import optuna
import numpy as np
import pandas as pd

from lightgbm import LGBMClassifier, LGBMRegressor, early_stopping, log_evaluation
from catboost import  CatBoostClassifier, CatBoostRegressor, Pool
from xgboost import XGBRegressor, XGBClassifier, callback

sys.path.append('../input/iterativestratification')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import StratifiedKFold, KFold

from datetime import datetime
from sklearn.decomposition import PCA

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

from lightgbm import early_stopping, log_evaluation
from catboost import Pool


class BaseGBDTClass:
    def __init__(self, model_class, params):
        self.model_class = model_class
        self.params = params

    def train(self, x_tr, y_tr):
        raise NotImplementedError("train method must be implemented")

    def train_with_validation(self, x_tr, y_tr, x_val, y_val):
        raise NotImplementedError("train_with_validation method must be implemented")

    def predict(self, model, input_):
        raise NotImplementedError("predict method must be implemented")

    def extract_importance(self, model, features, imp_col_prefix=None):
        importance = model.feature_importances_
        if imp_col_prefix is not None:
            importance_col = imp_col_prefix + "_importance"
        else:
            importance_col = "importance"
        importance_df = pl.DataFrame({"feature": features, importance_col: importance})
        return importance_df

    def check_pred_distribution_diff(self, predictions, gt, title):
        fig, ax = plt.subplots(figsize=(8, 5))
        sns.displot(predictions, ax=ax, label="prediction")
        sns.displot(gt, ax=ax, label="ground truth")
        ax.legend()
        ax.grid()
        ax.set_title(title)

    def test(self, models, test):
        test_predictions = [self.predict(model, test) for model in models]
        test_predictions = np.mean(test_predictions, axis=0)
        return test_predictions


class LGBClass(BaseGBDTClass):
    def __init__(self, cat_features="auto", custom_callback=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.cat_features = cat_features
        if custom_callback:
            self.callbacks = custom_callback
        else:
            self.callbacks = [
                early_stopping(stopping_rounds=50),
                log_evaluation(100),
            ]

    def train(self, x_tr, y_tr):

        model = self.model_class(**self.params)
        model.fit(
            x_tr,
            y_tr,
            categorical_feature=self.cat_features,
        )

        return model

    def train_with_validation(self, x_tr, y_tr, x_val, y_val):

        model = self.model_class(**self.params)
        model = model.fit(
            x_tr,
            y_tr,
            eval_set=[(x_val, y_val)],
            categorical_feature=self.cat_features,
            callbacks=self.callbacks,
        )
        return model

    def predict(self, model, input_):
        predictions = model.predict(input_)
        return predictions


class XGBClass(BaseGBDTClass):
    def __init__(self, verbose_eval_step: int, output_prob: bool, multi_label: bool, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.verbose_eval_step = verbose_eval_step
        self.output_prob = output_prob
        self.multi_label = multi_label

    def train(self, x_tr, y_tr):

        model = self.model_class(**self.params)
        model.fit(x_tr, y_tr)
        return model

    def train_with_validation(self, x_tr, y_tr, x_val, y_val):

        model = self.model_class(**self.params)
        model.fit(x_tr, y_tr, eval_set=[(x_val, y_val)], verbose=self.verbose_eval_step)
        return model

    def predict(self, model, input_):
        if self.output_prob:
            predictions = model.predict_proba(
                input_, iteration_range=(0, model.best_iteration)
            )
            if not self.multi_label:
                predictions = predictions[:, 1]
        else:
            predictions = model.predict(
                input_, iteration_range=(0, model.best_iteration)
            )
        return predictions


class CBClass(BaseGBDTClass):
    def __init__(self, output_prob: bool, multi_label: bool, cat=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.cat = cat
        self.multi_label = multi_label
        self.output_prob = output_prob

    def train(self, x_tr, y_tr):

        train_pool = Pool(data=x_tr, label=y_tr, cat_features=self.cat)

        model = self.model_class(**self.params)
        model.fit(train_pool)

        return model

    def train_with_validation(self, x_tr, y_tr, x_val, y_val):
        train_pool = Pool(data=x_tr, label=y_tr, cat_features=self.cat)
        valid_pool = Pool(data=x_val, label=y_val, cat_features=self.cat)

        model = self.model_class(**self.params)
        model.fit(
            train_pool,
            eval_set=[valid_pool],
            early_stopping_rounds=50,
            verbose_eval=100,
        )

        return model

    def predict(self, model, input_):
        if self.output_prob:
            predictions = model.predict_proba(input_)
            if not self.multi_label:
                predictions = predictions[:, 1]
        else:
            predictions = model.predict(input_)
        return predictions

    def extract_importance(self, model, features, imp_col_prefix=None):
        importance = model.get_feature_importance()
        if imp_col_prefix is not None:
            importance_col = imp_col_prefix + "_importance"
        else:
            importance_col = "importance"
        importance_df = pl.DataFrame({"feature": features, importance_col: importance})
        return importance_df


# preprocess

In [3]:
train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
sample = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')
greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')


train['EJ'] = train['EJ'].map({'A': 0, 'B': 1})
test['EJ']  = test['EJ'].map({'A': 0, 'B': 1})

# process epsilon
train = pd.merge(train, greeks, on = "Id", how = "inner")
train_stratify = train[["Class", "Beta", "Delta", "Gamma"]] 
train["Epsilon_ordinal"] = train["Epsilon"].map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal() if x != "Unknown" else np.nan)

org_features = [n for n in train.columns if n not in ['Class', 'Id', 'Alpha', "Beta", "Gamma", "Delta", "Epsilon"]]
test_times = pd.DataFrame([train.Epsilon_ordinal.max() + 1] * len(test), columns = ["Epsilon_ordinal"])
final_test = pd.concat((test, test_times), axis=1)

# fill missing value
train.fillna(-999, inplace=True)
final_test.fillna(-999, inplace=True)

# add pca columns
pca_feat_num = 15
pca_cols = ["pca"+str(i+1) for i in range(pca_feat_num)]
pca = PCA(n_components=pca_feat_num,random_state=42)
pca_train = pca.fit_transform(train[org_features])
pca_test = pca.transform(final_test[org_features])
pca_train = pd.DataFrame(pca_train, columns=pca_cols)
pca_test = pd.DataFrame(pca_test, columns=pca_cols)
train = pd.concat([train, pca_train],axis=1)
final_test = pd.concat([final_test, pca_test],axis=1)

target = train.Class
train = train[org_features + pca_cols]
final_test = final_test[org_features + pca_cols]

print(train.shape, final_test.shape)

(617, 72) (5, 72)


# config

In [4]:
from sklearn.metrics import log_loss
def balanced_log_loss(y_true, y_pred):
    nc = np.bincount(y_true)
    return log_loss(y_true, y_pred, sample_weight = 1/nc[y_true], eps=1e-15)

xgb_params = {
            'objective': 'binary:logistic',
            'eval_metric': 'logloss',
            'verbosity': 0,
            'seed': 42,
        }
    
lgb_params = {
            "n_estimators": 1000,
            'objective': 'binary', 
            'metric': 'binary_logloss', 
            'boosting': 'goss',
            'verbose': -1,
            'seed': 42,
            'is_unbalance': True
} 

cb_params = {
            'loss_function': 'Logloss',
            'iterations': 1000,  
            'random_seed': 42,
            'use_best_model' : True
}

In [5]:
cv_scheme = MultilabelStratifiedKFold(5, shuffle=True, random_state=42).split(train, train_stratify)
train_valid_idx_pairs = []
for split, (tr_idx, val_idx) in enumerate(cv_scheme): 
    train_valid_idx_pairs.append([tr_idx, val_idx])

# tree model

In [6]:
lgb_modelling = LGBClass(model_class = LGBMClassifier, params = lgb_params)
cb_modelling = CBClass(model_class = CatBoostClassifier, params = cb_params, output_prob = True, multi_label = False)
xgb_modelling = XGBClass(model_class = XGBClassifier, params = xgb_params, verbose_eval_step = 50, output_prob = True, multi_label = False)

## lgb

In [7]:
models = []
valid_output = np.zeros(len(target))
for split, (tr_idx, val_idx) in enumerate(train_valid_idx_pairs): 
    model = lgb_modelling.train_with_validation(train.loc[tr_idx], target.loc[tr_idx],
                                                          train.loc[val_idx], target.loc[val_idx])
    valid_output[val_idx] = lgb_modelling.predict(model, train.loc[val_idx])
    models.append(model)
val_score = balanced_log_loss(target, valid_output)
print("validation score: ", val_score)

lgb_test_output = lgb_modelling.test(models, final_test)

[LightGBM] [Warning] boosting is set=goss, boosting_type=gbdt will be ignored. Current value: boosting=goss
Training until validation scores don't improve for 50 rounds
[100]	valid_0's binary_logloss: 0.0753021
[200]	valid_0's binary_logloss: 0.0537239
Early stopping, best iteration is:
[198]	valid_0's binary_logloss: 0.0522064
[LightGBM] [Warning] boosting is set=goss, boosting_type=gbdt will be ignored. Current value: boosting=goss
[100]	valid_0's binary_logloss: 0.190418
[200]	valid_0's binary_logloss: 0.196041
Early stopping, best iteration is:
[198]	valid_0's binary_logloss: 0.0522064
[LightGBM] [Warning] boosting is set=goss, boosting_type=gbdt will be ignored. Current value: boosting=goss
[100]	valid_0's binary_logloss: 0.173883
[200]	valid_0's binary_logloss: 0.196299
Early stopping, best iteration is:
[198]	valid_0's binary_logloss: 0.0522064
[LightGBM] [Warning] boosting is set=goss, boosting_type=gbdt will be ignored. Current value: boosting=goss
[100]	valid_0's binary_loglo

## cb

In [8]:
models = []
valid_output = np.zeros(len(target))
for split, (tr_idx, val_idx) in enumerate(train_valid_idx_pairs): 
    model = cb_modelling.train_with_validation(train.loc[tr_idx], target.loc[tr_idx],
                                                    train.loc[val_idx], target.loc[val_idx])
    valid_output[val_idx] = cb_modelling.predict(model, train.loc[val_idx])
    models.append(model)
val_score = balanced_log_loss(target, valid_output)
print("validation score: ", val_score)
cb_test_output = cb_modelling.test(models, final_test)

Learning rate set to 0.026623
0:	learn: 0.6658456	test: 0.6657945	best: 0.6657945 (0)	total: 63ms	remaining: 1m 2s
100:	learn: 0.1123480	test: 0.1832610	best: 0.1832610 (100)	total: 612ms	remaining: 5.45s
200:	learn: 0.0492788	test: 0.1381917	best: 0.1381917 (200)	total: 1.16s	remaining: 4.61s
300:	learn: 0.0253975	test: 0.1166001	best: 0.1166001 (300)	total: 1.71s	remaining: 3.98s
400:	learn: 0.0143491	test: 0.1034707	best: 0.1034707 (400)	total: 2.36s	remaining: 3.52s
500:	learn: 0.0094645	test: 0.0988917	best: 0.0988917 (500)	total: 3.08s	remaining: 3.07s
600:	learn: 0.0068158	test: 0.0956670	best: 0.0952825 (577)	total: 3.7s	remaining: 2.45s
700:	learn: 0.0052091	test: 0.0923589	best: 0.0923589 (700)	total: 4.23s	remaining: 1.8s
800:	learn: 0.0042617	test: 0.0891180	best: 0.0891180 (800)	total: 4.75s	remaining: 1.18s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.08903141659
bestIteration = 803

Shrink model to first 804 iterations.
Learning rate set to 0.02662

# xgb

In [9]:
models = []
valid_output = np.zeros(len(target))
for split, (tr_idx, val_idx) in enumerate(train_valid_idx_pairs): 
    model = xgb_modelling.train_with_validation(train.loc[tr_idx], target.loc[tr_idx],
                                                        train.loc[val_idx], target.loc[val_idx])
    valid_output[val_idx] = xgb_modelling.predict(model, train.loc[val_idx])
    models.append(model)
val_score = balanced_log_loss(target, valid_output)
print("validation score: ", val_score)
xgb_test_output = xgb_modelling.test(models, final_test)

[0]	validation_0-logloss:0.50062
[50]	validation_0-logloss:0.08244
[99]	validation_0-logloss:0.07952
[0]	validation_0-logloss:0.52655
[50]	validation_0-logloss:0.15118
[99]	validation_0-logloss:0.15183
[0]	validation_0-logloss:0.50637
[50]	validation_0-logloss:0.16298
[99]	validation_0-logloss:0.16329
[0]	validation_0-logloss:0.52571
[50]	validation_0-logloss:0.15877
[99]	validation_0-logloss:0.15716
[0]	validation_0-logloss:0.52212
[50]	validation_0-logloss:0.20782
[99]	validation_0-logloss:0.21162
validation score:  0.3381245344172203


# tuning

In [10]:
def param_tuning(objective, trial_num = 5, option = "minimize"):
                        
    study = optuna.create_study(direction=option) 
    study.optimize(objective, n_trials=trial_num)
    trial = study.best_trial
    best_parameters = trial.params
    best_value = trial.value
    print('Value: ', best_value)
    print('best_parameters: ', best_parameters)

    optuna.visualization.plot_param_importances(study).show()

    return best_parameters, best_value

In [11]:
def lgb_objective(trial):

    check_params = {
                #"max_bin": trial.suggest_int("max_bin", 10, 500),
                'num_leaves': trial.suggest_int('num_leaves', 10, 100),
                #'min_data_in_leaf': trial.suggest_int("min_data_in_leaf", 2, 50),
                #'min_sum_hessian_in_leaf': trial.suggest_float('min_sum_hessian_in_leaf', 1e-8, 10.0, log=True),
                #'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
                'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
                #'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1.0),
                #'bagging_freq': trial.suggest_int('bagging_freq', 1, 100),
                #'min_child_samples': trial.suggest_int('min_child_samples', 1, 10),
                #'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0, 10),
                #'lambda_l2': trial.suggest_float('lambda_l2', 0, 0.1),
                #'lambda_l1': trial.suggest_float('lambda_l1', 0, 0.1),
                #'max_depth': trial.suggest_int('max_depth', 2, 100),
                #"extra_trees": trial.suggest_categorical('extra_trees', [True, False]),
                #"path_smooth": trial.suggest_int("path_smooth", 0, 10),
    }
    new_params = lgb_params.copy()
    new_params.update(check_params)

    model_class = LGBClass(model_class = LGBMClassifier, params = new_params)

    valid_output = np.zeros(len(target))
    for split, (tr_idx, val_idx) in enumerate(train_valid_idx_pairs): 
        model = model_class.train_with_validation(train.loc[tr_idx], target.loc[tr_idx],
                                                        train.loc[val_idx], target.loc[val_idx])
        valid_output[val_idx] = model_class.predict(model, train.loc[val_idx])
    score = balanced_log_loss(target, valid_output)
    return score

lgb_best_params, lgb_best_value = param_tuning(lgb_objective)
lgb_best_params, lgb_best_value

[I 2025-04-03 00:10:39,613] A new study created in memory with name: no-name-72c9bc96-8a06-4776-b61c-c325554ae943


[LightGBM] [Warning] boosting is set=goss, boosting_type=gbdt will be ignored. Current value: boosting=goss
[LightGBM] [Warning] feature_fraction is set=0.8390209808447286, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8390209808447286
Training until validation scores don't improve for 50 rounds
[100]	valid_0's binary_logloss: 0.0791658
[200]	valid_0's binary_logloss: 0.0521942
[300]	valid_0's binary_logloss: 0.052448
[400]	valid_0's binary_logloss: 0.048403
Early stopping, best iteration is:
[370]	valid_0's binary_logloss: 0.0457814
[LightGBM] [Warning] boosting is set=goss, boosting_type=gbdt will be ignored. Current value: boosting=goss
[LightGBM] [Warning] feature_fraction is set=0.8390209808447286, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8390209808447286
[100]	valid_0's binary_logloss: 0.184643
[200]	valid_0's binary_logloss: 0.182337
[300]	valid_0's binary_logloss: 0.199001
[400]	valid_0's binary_logloss: 0.235876
Early st

[I 2025-04-03 00:10:42,780] Trial 0 finished with value: 2.9995030165666097 and parameters: {'num_leaves': 13, 'feature_fraction': 0.8390209808447286}. Best is trial 0 with value: 2.9995030165666097.


[300]	valid_0's binary_logloss: 0.215584
[400]	valid_0's binary_logloss: 0.271553
Early stopping, best iteration is:
[370]	valid_0's binary_logloss: 0.0457814
[LightGBM] [Warning] boosting is set=goss, boosting_type=gbdt will be ignored. Current value: boosting=goss
[LightGBM] [Warning] feature_fraction is set=0.6657267624202725, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6657267624202725
Training until validation scores don't improve for 50 rounds
[100]	valid_0's binary_logloss: 0.079783
[200]	valid_0's binary_logloss: 0.048642
[300]	valid_0's binary_logloss: 0.036067
[400]	valid_0's binary_logloss: 0.0344897
Early stopping, best iteration is:
[369]	valid_0's binary_logloss: 0.0299517
[LightGBM] [Warning] boosting is set=goss, boosting_type=gbdt will be ignored. Current value: boosting=goss
[LightGBM] [Warning] feature_fraction is set=0.6657267624202725, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6657267624202725
[100]	valid_0'

[I 2025-04-03 00:10:45,811] Trial 1 finished with value: 3.0092455568676226 and parameters: {'num_leaves': 23, 'feature_fraction': 0.6657267624202725}. Best is trial 0 with value: 2.9995030165666097.


...

[I 2025-04-03 00:10:51,210] Trial 3 finished with value: 3.1594047591356405 and parameters: {'num_leaves': 30, 'feature_fraction': 0.67948681787784}. Best is trial 0 with value: 2.9995030165666097.


[300]	valid_0's binary_logloss: 0.205316
Early stopping, best iteration is:
[293]	valid_0's binary_logloss: 0.0321089
[LightGBM] [Warning] boosting is set=goss, boosting_type=gbdt will be ignored. Current value: boosting=goss
[LightGBM] [Warning] feature_fraction is set=0.9447897560867842, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9447897560867842
Training until validation scores don't improve for 50 rounds
[100]	valid_0's binary_logloss: 0.073851
[200]	valid_0's binary_logloss: 0.0472328
[300]	valid_0's binary_logloss: 0.0406845
Early stopping, best iteration is:
[263]	valid_0's binary_logloss: 0.0367877
[LightGBM] [Warning] boosting is set=goss, boosting_type=gbdt will be ignored. Current value: boosting=goss
[LightGBM] [Warning] feature_fraction is set=0.9447897560867842, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9447897560867842
[100]	valid_0's binary_logloss: 0.183165
[200]	valid_0's binary_logloss: 0.19847
[300]	valid_0'

[I 2025-04-03 00:10:53,745] Trial 4 finished with value: 3.0092455568676226 and parameters: {'num_leaves': 49, 'feature_fraction': 0.9447897560867842}. Best is trial 0 with value: 2.9995030165666097.


[300]	valid_0's binary_logloss: 0.204401
Early stopping, best iteration is:
[263]	valid_0's binary_logloss: 0.0367877
Value:  2.9995030165666097
best_parameters:  {'num_leaves': 13, 'feature_fraction': 0.8390209808447286}


({'num_leaves': 13, 'feature_fraction': 0.8390209808447286},
 2.9995030165666097)

In [12]:
def xgb_objective(trial):

    check_params = {
                "verbosity": 0,
                "gamma": trial.suggest_float("gamma", 0, 1),
                #"max_depth": trial.suggest_int("max_depth", 1, 10),
                #"num_leaves": trial.suggest_int("num_leaves", 10, 100),
                #"learning_rate": trial.suggest_float("learning_rate", 0.01, 0.5),
                #"min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
                #"max_delta_step": trial.suggest_int("max_delta_step", 0, 10),
                #"subsample": trial.suggest_float("subsample", 0, 1),
                #"lambda": trial.suggest_float("lambda", 0, 1),
                #"alpha": trial.suggest_float("alpha", 0, 1),
            }

    new_params = xgb_params.copy()
    new_params.update(check_params)

    model_class = XGBClass(model_class = XGBClassifier, params = new_params,
                           verbose_eval_step = 50, output_prob = True, multi_label = False)

    valid_output = np.zeros(len(target))
    for split, (tr_idx, val_idx) in enumerate(train_valid_idx_pairs): 
        model = model_class.train_with_validation(train.loc[tr_idx], target.loc[tr_idx],
                                                        train.loc[val_idx], target.loc[val_idx])
        valid_output[val_idx] = model_class.predict(model, train.loc[val_idx])
    score = balanced_log_loss(target, valid_output)
    return score

xgb_best_params, xgb_best_value = param_tuning(xgb_objective)
xgb_best_params, xgb_best_value

[I 2025-04-03 00:10:54,382] A new study created in memory with name: no-name-cff3565b-e9b2-48b7-80d7-a4cc5576308f


[0]	validation_0-logloss:0.50132
[50]	validation_0-logloss:0.10666
[99]	validation_0-logloss:0.10666
[0]	validation_0-logloss:0.52655
[50]	validation_0-logloss:0.16041
[99]	validation_0-logloss:0.16041
[0]	validation_0-logloss:0.50315
[50]	validation_0-logloss:0.17950
[99]	validation_0-logloss:0.17950
[0]	validation_0-logloss:0.52638
[50]	validation_0-logloss:0.16462
[99]	validation_0-logloss:0.16462
[0]	validation_0-logloss:0.52017
[50]	validation_0-logloss:0.19874
[99]	validation_0-logloss:0.19874


[I 2025-04-03 00:10:56,017] Trial 0 finished with value: 0.32614449311622123 and parameters: {'gamma': 0.9198787882664067}. Best is trial 0 with value: 0.32614449311622123.


[0]	validation_0-logloss:0.50132
[50]	validation_0-logloss:0.09916
[99]	validation_0-logloss:0.09916
[0]	validation_0-logloss:0.52655
[50]	validation_0-logloss:0.15683
[99]	validation_0-logloss:0.15683
[0]	validation_0-logloss:0.50569
[50]	validation_0-logloss:0.16340
[99]	validation_0-logloss:0.16340
[0]	validation_0-logloss:0.52638
[50]	validation_0-logloss:0.18021
[99]	validation_0-logloss:0.18021
[0]	validation_0-logloss:0.52001
[50]	validation_0-logloss:0.20418
[99]	validation_0-logloss:0.20418


[I 2025-04-03 00:10:57,452] Trial 1 finished with value: 0.33799175134293097 and parameters: {'gamma': 0.5226972472816125}. Best is trial 0 with value: 0.32614449311622123.


...

[I 2025-04-03 00:11:00,722] Trial 3 finished with value: 0.32418928150930176 and parameters: {'gamma': 0.21824823772960078}. Best is trial 2 with value: 0.32248015069841846.


[0]	validation_0-logloss:0.50132
[50]	validation_0-logloss:0.09507
[99]	validation_0-logloss:0.09507
[0]	validation_0-logloss:0.52655
[50]	validation_0-logloss:0.16543
[99]	validation_0-logloss:0.16543
[0]	validation_0-logloss:0.50569
[50]	validation_0-logloss:0.15788
[99]	validation_0-logloss:0.15788
[0]	validation_0-logloss:0.52638
[50]	validation_0-logloss:0.18752
[99]	validation_0-logloss:0.18752
[0]	validation_0-logloss:0.52001
[50]	validation_0-logloss:0.19418
[99]	validation_0-logloss:0.19418


[I 2025-04-03 00:11:02,197] Trial 4 finished with value: 0.3432741459670619 and parameters: {'gamma': 0.5771753453149779}. Best is trial 2 with value: 0.32248015069841846.


Value:  0.32248015069841846
best_parameters:  {'gamma': 0.8968416781442157}


({'gamma': 0.8968416781442157}, 0.32248015069841846)

In [13]:
def cb_objective(trial):

    check_params = {
                "verbose": False,
                "max_depth": trial.suggest_int("max_depth", 1, 10),
                # 'num_leaves': trial.suggest_int('num_leaves', 10, 100),
                "reg_lambda": trial.suggest_float("reg_lambda", 0, 1.0),
                "learning_rate": trial.suggest_float("learning_rate", 0.05, 0.5),
                "min_child_samples": trial.suggest_int("min_child_samples", 1, 10),
                "colsample_bylevel": trial.suggest_float("reg_lambda", 0, 1.0),
            }

    new_params = cb_params.copy()
    new_params.update(check_params)

    model_class = CBClass(model_class = CatBoostClassifier, params = new_params,
                          output_prob = True, multi_label = False)

    valid_output = np.zeros(len(target))
    for split, (tr_idx, val_idx) in enumerate(train_valid_idx_pairs): 
        model = model_class.train_with_validation(train.loc[tr_idx], target.loc[tr_idx],
                                                        train.loc[val_idx], target.loc[val_idx])
        valid_output[val_idx] = model_class.predict(model, train.loc[val_idx])
    score = balanced_log_loss(target, valid_output)
    return score

cb_best_params, cb_best_value = param_tuning(cb_objective)
cb_best_params, cb_best_value

[I 2025-04-03 00:11:02,321] A new study created in memory with name: no-name-29da6424-ca49-4223-9250-db9d81d298b5


0:	learn: 0.4984757	test: 0.5664083	best: 0.5664083 (0)	total: 6.39ms	remaining: 6.38s
100:	learn: 0.0002798	test: 0.1811736	best: 0.1786160 (55)	total: 830ms	remaining: 7.39s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.1786159814
bestIteration = 55

Shrink model to first 56 iterations.
0:	learn: 0.4597054	test: 0.5544622	best: 0.5544622 (0)	total: 6.38ms	remaining: 6.38s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.2969948768
bestIteration = 7

Shrink model to first 8 iterations.
0:	learn: 0.4724080	test: 0.5663967	best: 0.5663967 (0)	total: 7.68ms	remaining: 7.67s
100:	learn: 0.0002668	test: 0.2548191	best: 0.2537329 (75)	total: 849ms	remaining: 7.55s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.2537329378
bestIteration = 75

Shrink model to first 76 iterations.
0:	learn: 0.4758794	test: 0.5603418	best: 0.5603418 (0)	total: 6.4ms	remaining: 6.39s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.2

[I 2025-04-03 00:11:06,334] Trial 0 finished with value: 0.636829604573982 and parameters: {'max_depth': 9, 'reg_lambda': 0.06558557336612225, 'learning_rate': 0.18992657440179556, 'min_child_samples': 4}. Best is trial 0 with value: 0.636829604573982.


Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.2529983673
bestIteration = 20

Shrink model to first 21 iterations.
0:	learn: 0.4694879	test: 0.4796736	best: 0.4796736 (0)	total: 5.71ms	remaining: 5.7s
100:	learn: 0.0016273	test: 0.0951723	best: 0.0945766 (94)	total: 652ms	remaining: 5.8s
200:	learn: 0.0006874	test: 0.0854599	best: 0.0852821 (174)	total: 1.27s	remaining: 5.03s
300:	learn: 0.0006587	test: 0.0838676	best: 0.0838676 (300)	total: 1.86s	remaining: 4.32s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.08373223789
bestIteration = 320

Shrink model to first 321 iterations.
0:	learn: 0.4827983	test: 0.5091058	best: 0.5091058 (0)	total: 5.63ms	remaining: 5.63s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.1990574917
bestIteration = 27

Shrink model to first 28 iterations.
0:	learn: 0.5089822	test: 0.5163548	best: 0.5163548 (0)	total: 5.66ms	remaining: 5.66s
Stopped by overfitting detector  (50 iterations wait)

bestTe

[I 2025-04-03 00:11:11,055] Trial 1 finished with value: 0.36365609246088937 and parameters: {'max_depth': 6, 'reg_lambda': 0.9655789453975407, 'learning_rate': 0.19935738452046448, 'min_child_samples': 4}. Best is trial 1 with value: 0.36365609246088937.


...

[I 2025-04-03 00:11:12,362] Trial 3 finished with value: 0.5653220579740393 and parameters: {'max_depth': 6, 'reg_lambda': 0.05185625114089987, 'learning_rate': 0.4913774936310952, 'min_child_samples': 1}. Best is trial 1 with value: 0.36365609246088937.


0:	learn: 0.3583841	test: 0.3966307	best: 0.3966307 (0)	total: 1.06ms	remaining: 1.06s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.3143374995
bestIteration = 7

Shrink model to first 8 iterations.
0:	learn: 0.5510414	test: 0.5537821	best: 0.5537821 (0)	total: 434us	remaining: 434ms
100:	learn: 0.0005842	test: 0.1008399	best: 0.0975282 (94)	total: 49.6ms	remaining: 441ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.08501737461
bestIteration = 128

Shrink model to first 129 iterations.
0:	learn: 0.5509419	test: 0.5547073	best: 0.5547073 (0)	total: 417us	remaining: 417ms
100:	learn: 0.0004101	test: 0.2555394	best: 0.2028993 (58)	total: 48.9ms	remaining: 435ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.2028992756
bestIteration = 58

Shrink model to first 59 iterations.
0:	learn: 0.5509276	test: 0.5536428	best: 0.5536428 (0)	total: 452us	remaining: 452ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 

[I 2025-04-03 00:11:12,855] Trial 4 finished with value: 0.47670468992774134 and parameters: {'max_depth': 4, 'reg_lambda': 0.03563447809368614, 'learning_rate': 0.3538672981615714, 'min_child_samples': 6}. Best is trial 1 with value: 0.36365609246088937.


100:	learn: 0.0003943	test: 0.2279019	best: 0.2123162 (90)	total: 48.2ms	remaining: 429ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.2123161829
bestIteration = 90

Shrink model to first 91 iterations.
0:	learn: 0.5529363	test: 0.5494602	best: 0.5494602 (0)	total: 409us	remaining: 409ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.3127574403
bestIteration = 9

Shrink model to first 10 iterations.
Value:  0.36365609246088937
best_parameters:  {'max_depth': 6, 'reg_lambda': 0.9655789453975407, 'learning_rate': 0.19935738452046448, 'min_child_samples': 4}


({'max_depth': 6,
  'reg_lambda': 0.9655789453975407,
  'learning_rate': 0.19935738452046448,
  'min_child_samples': 4},
 0.36365609246088937)